# Hartree and Hartree-Fock approximations.<br>


### In this demo we are going to compare the Hartree and the Hartree-Fock approximations for computing the molecular energy and check the Koopmans theorem. <br>


### Let's import some modules.

In [1]:
# imports
import chemlab
import chemview
import numpy as np 
from scipy.linalg import eigh
import scipy.linalg as LA
import matplotlib.pyplot as plt
# Pyquante
import PyQuante
from PyQuante.Molecule import Molecule
from PyQuante import Ints
from PyQuante.Ints import coulomb
from PyQuante.hartree_fock import rhf
from PyQuante.Ints import getbasis
from PyQuante import SCF
from PyQuante import configure_output
# add all probables spots for the quantum world library
import sys
sys.path.append('../library')
sys.path.append('../../library')
sys.path.append('/home/student/chem160/library')
#This is how we'll import our own home-made modules
import quantumWorld as qworld
import hartreeProduct as hp
%matplotlib inline

/home/student/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/student/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


# <i class="fa fa-wrench"></i> Step 1 : <i class="fa fa-wrench"></i>  Define the molecule.<br>

### We are going to calculate the hydrogen molecule with an internuclear distance of 0.75 angstroms. Create the molecule using the Molecule object from pyquante:
```python
variable_name = Molecule('name',
                          atomlist = [(atomic_number,(x1, y1, z1)),
                                      (atomic_number,(x2, y2, z2))],
                          units='Angstrom')
```

In [2]:
r = 0.75
# COMPLETE THE CODE TO CREATE MOLECULAR HYDROGEN

## Now visualize it:

In [3]:
qworld.visualize_Mol(h2)

# <i class="fa fa-wrench"></i> Step 2 : Define the basis and run the Hartree-Fock calculation.

Recall from class that our Hartree Fock energy is given by:

$$E = \sum_{i=1}^N\left( h_i + \epsilon_i \right)$$

where the *orbital energies*, $\epsilon_i$ are given by:

$$\epsilon_i = h_i + \sum_{j=1}^N \left( 2 J_{ij} - K_{ij} \right), \ \text{(Closed shell molecule)} $$ 

where

$$h_j = \int dr_j \psi^\dagger_j(r_j) \left( -\frac{1}{2}\nabla^2_j - \sum_A^M \frac{Z_A}{R_{jA}}  \right) \psi_j(r_j) $$

is the 1 electron integral,

$$J_{ij} = \int \int dr_1 dr_2 \psi^*_i(r_1) \psi^*_j(r_2) \frac{1}{r_{12}} \psi_i(r_1) \psi_j(r_2) $$

is the coulomb integral, and

$$K_{ij} = \int \int dr_1 dr_2 \psi^*_i(r_1) \psi^*_j(r_2) \frac{1}{r_{12}} \psi_i(r_2) \psi_j(r_1) $$
is the exchange integral.

You're going to write this in your considerable exercise, but for the time being, we'll just go ahead and use PyQuante. The method that you're interested in is Restricted Hartree-Fock (rhf)
```python
    HFenergy, orbital_energies, orbitals = rhf(molecule,basis_data=basis_set)
```

In [4]:
basis_set='6-31G**'
configure_output(stream=sys.stderr)
# COMPLETE THE LINE TO RUN THE RESTRICTED HARTREE-FOCK CALCULATION USING PYQUANTE


print "FINAL HF ENERGY:"
print HFenergy

RHF calculation on h2
Nbf = 10
Nclosed = 1
Calculating Integrals...
Integrals Calculated.
Using DIIS averaging
Iteration: 0    Energy: -1.069550    EnergyVar: 1.069550
Iteration: 1    Energy: -1.137402    EnergyVar: 0.067852
Iteration: 2    Energy: -1.131095    EnergyVar: 0.006307
Iteration: 3    Energy: -1.131128    EnergyVar: 0.000034
PyQuante converged in 4 iterations
Final HF energy for system h2 is -1.131119


FINAL HF ENERGY:
-1.13111938387


# <i class="fa fa-wrench"></i>  Step 3: Now let's explore some properties: the Koopmans' theorem. 

### Next, let's go ahead and see if Koopman's theorem also applies here. We know that the experimental first ionization energy of hydrogen is $15.43eV$. Where $1 \ ha=27.211eV$. <br>

### Compute the ionization energy of molecular hydrogen in eV using the Koopmans' theorem. Remember that you obtained the orbital energies from the HF calculation.

In [5]:
firstIEH2=
print "The ionization energy of H2 is: ",firstIEH2,"eV"
print "using the Koopmans' approach with the 6-31G** basis set." 
print "The percentage error is:",100*abs(firstIEH2-15.43)/15.43


The ionization energy of H2 is:  16.1049760778 eV
using the Koopmans' approach with the 6-31G** basis set.
The percentage error is: 4.37443990803


## ... Curious fact <i class="fa fa-lightbulb-o"></i>

### The Koopmans' theorem is named after Tjalling Koopmans, who published this result in 1934. Tjalling started as a student in mathematics, then moved to theoretical physics and finally to economics. He is famous in the quantum chemistry field because of his theorem, but guess what, he is even more famous in economics, where he got the nobel prize for his contributions to the theory of optimal use of resources. Well done Tjalling.

![](files/koopmans.jpg)


# <i class="fa fa-wrench"></i> Step 4: Run the Hartree product calculation, how bad is it? 

Recall from class that the energy of the Hartree-Product approximation is found as:

$$ E_0 = \langle \psi_0 | \sum_i \hat{h}_i + \sum_{i>j} \frac{1}{| r_i - r_j |} | \psi_0 \rangle,$$
where
$$\psi_T = |\phi_1^1 \rangle |\phi_2^2 \rangle ... |\phi_N^N \rangle.$$

This simplifies to:

$$E_T = \sum_i \langle \phi_i | \hat{h}_i | \phi_i \rangle + \sum_{i>j}  ( ij | ij ).$$

We have implemented the Hartree-Product using our own routine, so all you need to do is call the **hp.hartreeProduct()** method and you'll be all set:

    HPenergy, orbital_energies, MO_coeff = hp.hartreeProduct(molecule,basis=basis_name)

In [6]:
# COMPLETE THE LINE FOR THE HARTREE-PRODUCT CALCULATION

print "FINAL Hartree-Product ENERGY:"
print HPenergy

Iteration number 1, energy = -0.427649, energyvar = 0.427649
Iteration number 2, energy = -0.497845, energyvar = 0.070196
Iteration number 3, energy = -0.515634, energyvar = 0.017789
Iteration number 4, energy = -0.520979, energyvar = 0.005345
Iteration number 5, energy = -0.522566, energyvar = 0.001587
Iteration number 6, energy = -0.523047, energyvar = 0.000481
Converged in 6 iterations
Final Hartree Fock Energy for h2 is -0.523047
FINAL Hartree-Product ENERGY:
-0.523046518583
